## Direct API calls to Gemini

In [1]:
# import base64
import os
from google import genai
# from google.genai import types


def generate(prompt):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[prompt]
    )

    print(response.text)
    

generate("What is 1 + 3?")


1 + 3 = 4



In [2]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)


Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [3]:
generate(prompt)

Oh dear, I'm quite frustrated because my blender lid came off and splashed smoothie all over my kitchen walls! To add to that, the warranty unfortunately doesn't cover the cost of cleaning up my kitchen. I could really use some assistance right now, if you don't mind.



## Chat API : LangChain

In [8]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = os.environ["GEMINI_API_KEY"]

gemini = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [9]:
from langchain_core.prompts import PromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

# Instantiation using from_template (recommended)
prompt_template = PromptTemplate.from_template(template_string)

print(prompt_template.template)
print(prompt_template.input_variables)
print(prompt_template)

Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```

['style', 'text']
input_variables=['style', 'text'] input_types={} partial_variables={} template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'


In [10]:
customer_style = """Super Polite English Gentleman"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = prompt_template.format_prompt(
                    style=customer_style,
                    text=customer_email)

print(type(customer_messages))
print(customer_messages)

<class 'langchain_core.prompt_values.StringPromptValue'>
text="Translate the text that is delimited by triple backticks into a style that is Super Polite English Gentleman. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n"


In [11]:
customer_response = gemini.invoke(customer_messages)
print(customer_response)

content='Right then, I must confess to being rather perturbed, indeed quite cross, that the lid of my, ah, *blender* did, in a most unfortunate manner, become detached during operation, resulting in a rather extensive and, dare I say, messy distribution of smoothie across the kitchen walls. To compound this regrettable situation, the warranty, it appears, does not extend to the defraying of expenses associated with the subsequent cleaning. I find myself in a rather delicate predicament, and would be most grateful for your assistance at this very moment, old chap!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-ef462ed6-3c11-40eb-b2d7-72cca9e7337f-0' usage_metadata={'input_tokens': 79, 'output_tokens': 112, 'total_tokens': 191, 'input_token_details': {'cache_read': 0}}


In [12]:
customer_response = gemini.invoke("Translate the text that is delimited by triple backticks into a style that is Super Polite English Gentleman. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n")

In [13]:
print(customer_response)

content='Right then, I must confess to being rather perturbed, indeed quite cross, that the lid of my, ah, *blender* did, in a most unfortunate manner, become detached during operation, resulting in a rather extensive and, dare I say, messy distribution of smoothie across the kitchen walls. To compound this regrettable situation, the warranty, it appears, does not extend to the defraying of expenses associated with the subsequent cleaning. I find myself in a rather delicate predicament, and would be most grateful for your assistance at this very moment, old chap!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-237ab02b-7a9f-4dd3-bc38-67656c9e70b4-0' usage_metadata={'input_tokens': 79, 'output_tokens': 112, 'total_tokens': 191, 'input_token_details': {'cache_read': 0}}


In [14]:
messages = [
    ("system", "Translate the text that is delimited by triple backticks into a style that is Super Polite English Gentleman."),
    ("human", "```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```."),
]
gemini.invoke(messages).content


"Right then, I must confess that I find myself in a rather unfortunate predicament. It appears that my blender's lid, in a most uncouth manner, decided to detach itself during operation, resulting in a rather messy distribution of smoothie across the kitchen walls. To add insult to injury, the warranty, it seems, does not extend its protection to the expenses associated with restoring the kitchen to its former pristine condition. I would be most grateful if you could lend me your assistance in this matter, old chap!"

In [15]:
from langchain_core.prompts import ChatPromptTemplate

chat_template_string = [
    ("system", "Translate the text that is delimited by triple backticks into a style that is {style}."),
    ("human", "```{text}```"),
]
chat_prompt_template = ChatPromptTemplate(chat_template_string)
customer_messages = chat_prompt_template.invoke(
                        {
                            "style": customer_style,
                            "text": customer_email,
                        }
                    )
print(customer_messages.messages)
gemini.invoke(customer_messages).content

[SystemMessage(content='Translate the text that is delimited by triple backticks into a style that is Super Polite English Gentleman.', additional_kwargs={}, response_metadata={}), HumanMessage(content="```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```", additional_kwargs={}, response_metadata={})]


"Good heavens! I must confess, I find myself in a rather unfortunate predicament. It appears that my blender's lid, in a most uncouth manner, decided to detach itself during operation, resulting in a rather messy distribution of smoothie across the kitchen walls. To add insult to injury, the warranty, it seems, does not extend its protection to the cost of rectifying this… this… *splatter*. I find myself in dire need of assistance, my good fellow! Perhaps you might be able to lend a hand?"

In [13]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_messages = prompt_template.format_prompt(
    style=service_style_pirate,
    text=service_reply)

print(service_messages)

text="Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!\n```\n"


In [14]:
service_response = gemini.invoke(service_messages)
print(service_response)

content="Ahoy there, matey!\n\nI be havin' to inform ye that, alas, the warranty be not coverin' the cleanin' expenses for yer galley. It seems a bit o' a blunder occurred with yer blender, and 'tis yer own doin' for forgettin' to secure the lid afore givin' 'er a whirl. A bit o' bad luck, I'm afraid! Fair winds to ye!" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-c94d0d8f-ed79-42bd-9470-7c25a8014f6f-0' usage_metadata={'input_tokens': 73, 'output_tokens': 91, 'total_tokens': 164, 'input_token_details': {'cache_read': 0}}


In [15]:
service_messages = chat_prompt_template.invoke(
                        {
                            "style": service_style_pirate,
                            "text": service_reply,
                        }
                    )
service_response = gemini.invoke(service_messages)
print(service_response)

content="Ahoy there, matey!\n\nRegarding yer kitchen, I be afraid the warranty doesn't cover the cleaning expenses. It seems the mishap with yer blender, lackin' a lid afore startin' it, be the cause o' the mess. A bit o' misfortune on yer part, I'd say. Fair winds to ye!" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-fa750c18-240f-4915-9843-bb95514534c0-0' usage_metadata={'input_tokens': 70, 'output_tokens': 72, 'total_tokens': 142, 'input_token_details': {'cache_read': 0}}


## Output Parsers

In [17]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [24]:
messages = prompt_template.format_messages(text=customer_review)
model = "gemma-3-27b-it" # "gemini-2.0-flash-lite"
chat = ChatGoogleGenerativeAI(
    model=model,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)
response = chat.invoke(messages)
print(response.content)

```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there",
    "I think it's worth it for the extra features"
  ]
}
```


In [25]:
type(response.content)

str

In [26]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [27]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [28]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [29]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [30]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [31]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [32]:
response = chat(messages)
print(response.content)

```json
{
	"gift": "True",
	"delivery_days": "2",
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [33]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [34]:
type(output_dict)

dict

In [35]:
output_dict.get('delivery_days')

'2'

In [38]:
output_dict.get('price_value')

"It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."